In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import random
import plotly.express as px

In [2]:
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt


In [3]:
df = pd.read_parquet("Shopping Carts.parquet")
df.reset_index(inplace=True)

In [50]:
df

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo,8
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo,8
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo,8
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4


### Prüfen Sie, ob es beim Produkt 13176 auffällige Muster beim Wiederbestellverhalten gibt. 

In [4]:
df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index()
df = pd.merge(df,df.groupby("order_id").agg(order_size=("add_to_cart_order","max")).reset_index())

In [5]:
df_auf1 = df[df.product_id == 13176]

In [6]:
liste = df_auf1.groupby("user_id").agg(order_id=("order_number","min")).reset_index().order_id.to_list()
df_auf1.loc[df_auf1.order_id.isin(liste),"reordered"] = 0

pro county 

In [51]:
df_auf1

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo,8
23,3482059,6,13176,3,0,89125,11,0,20,3.0,0,Bag of Organic Bananas,24,4,produce,fresh fruits,Sacramento,3
134,1726138,18,13176,4,0,56463,41,3,8,14.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Calaveras,49
296,4560808,29,13176,1,0,103040,12,1,10,7.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Yuba,5
460,1912128,47,13176,10,0,143768,33,4,23,2.0,0,Bag of Organic Bananas,24,4,produce,fresh fruits,Los Angeles,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132314,2454605,606882,13176,12,0,137664,10,0,14,27.0,0,Bag of Organic Bananas,24,4,produce,fresh fruits,Fresno,24
6132363,3629290,606885,13176,1,1,32829,2,6,15,11.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Siskiyou,14
6132372,4211553,606886,13176,4,1,19992,2,6,18,6.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Marin,9
6132455,3961519,606894,13176,6,1,38426,14,3,18,21.0,0,Bag of Organic Bananas,24,4,produce,fresh fruits,San Mateo,12


In [39]:
user = df_auf1[df_auf1.reordered == 1].user_id.unique()

In [40]:
df_user = df[df.user_id.isin(user)]

In [41]:
banana_order = df_auf1.order_id.unique().tolist()

In [42]:
df_user["Banana"] = False
df_user.loc[df_user.order_id.isin(banana_order),"Banana"] = True


C:\Users\paulh\AppData\Local\Temp\ipykernel_20872\558051902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user["Banana"] = False


In [32]:
df_user

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size,Banana
8,3638166,2,41897,3,0,22352,4,1,12,30.0,0,Clean Day Lavender Scent Room Freshener Spray,101,17,household,air fresheners candles,Calaveras,3,False
9,3638165,2,15873,2,0,22352,4,1,12,30.0,0,Dryer Sheets Geranium Scent,75,17,household,laundry,Calaveras,3,False
10,3638164,2,40462,1,0,22352,4,1,12,30.0,0,Cleanse,31,7,beverages,refrigerated,Calaveras,3,False
16,2538684,4,40078,1,0,193635,19,0,14,0.0,1,Strawberry Lemonade Frozen Pops,37,1,frozen,ice cream ice,Glenn,3,False
17,2538686,4,46175,3,0,193635,19,0,14,0.0,1,Half Baked® Ice Cream,37,1,frozen,ice cream ice,Glenn,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132664,2493330,606914,28849,6,1,193225,18,5,13,4.0,1,No Salt Added Black Beans,59,15,canned goods,canned meals beans,Calaveras,8,False
6132665,3597250,606915,27966,1,0,107102,1,3,10,NaN,0,Organic Raspberries,123,4,produce,packaged vegetables fruits,Amador,4,False
6132666,3597251,606915,27845,4,0,107102,1,3,10,NaN,0,Organic Whole Milk,84,16,dairy eggs,milk,Amador,4,False
6132667,3597252,606915,26940,3,0,107102,1,3,10,NaN,0,Organic Large Green Asparagus,83,4,produce,fresh vegetables,Amador,4,False


In [43]:
df_test2 = df_user[df_user.product_id == 13176][["user_id","order_number","add_to_cart_order"]]

In [51]:
df_user1 = df_user.groupby(["user_id","order_number"])[["Banana","order_size","days_since_prior_order","order_dow","tip","order_hour_of_day"]].max().reset_index()

In [52]:
df_user1 = pd.merge(df_user1,df_test2,how="left")

In [54]:
df_user1.head(50)

,user_id,order_number,Banana,order_size,days_since_prior_order,order_dow,tip,order_hour_of_day,add_to_cart_order
0,40,1,True,12,NaN,0,1,9,1.0
1,40,2,True,13,8.0,1,0,10,2.0
2,40,3,False,15,12.0,6,0,11,NaN
3,40,4,False,11,8.0,0,1,12,NaN
4,40,5,True,11,20.0,6,0,9,4.0
5,40,6,True,9,15.0,0,1,9,5.0
6,40,7,True,11,21.0,0,0,8,5.0
7,40,8,True,10,7.0,0,1,10,2.0
8,40,9,True,12,14.0,0,1,9,2.0
9,42,1,False,5,NaN,2,0,10,NaN


In [55]:
df_user1[df_user1.Banana == True].head(50)

,user_id,order_number,Banana,order_size,days_since_prior_order,order_dow,tip,order_hour_of_day,add_to_cart_order
0,40,1,True,12,NaN,0,1,9,1.0
1,40,2,True,13,8.0,1,0,10,2.0
4,40,5,True,11,20.0,6,0,9,4.0
5,40,6,True,9,15.0,0,1,9,5.0
6,40,7,True,11,21.0,0,0,8,5.0
7,40,8,True,10,7.0,0,1,10,2.0
8,40,9,True,12,14.0,0,1,9,2.0
15,42,7,True,28,9.0,3,0,13,25.0
16,42,8,True,14,11.0,0,0,10,3.0
26,65,1,True,16,NaN,0,0,9,11.0


: 

In [50]:
px.scatter(df_user1[df_user1.Banana == True].groupby("order_size").agg(coun=("add_to_cart_order","median")))

In [20]:
df_user1[df_user1.order_number.mod(2) == 0].Banana.sum()

32582

In [23]:
df_user1[df_user1.order_number.mod(2) == 0].count()

user_id                   101925
order_number              101925
Banana                    101925
order_size                101925
days_since_prior_order    101925
order_dow                 101925
tip                       101925
dtype: int64

In [24]:
32582/101925

0.31966642138827567

In [26]:
df_user1[df_user1.order_number.mod(3) == 0].Banana.sum()

21184

In [25]:
df_user1[df_user1.order_number.mod(3) == 0].count()

user_id                   66545
order_number              66545
Banana                    66545
order_size                66545
days_since_prior_order    66545
order_dow                 66545
tip                       66545
dtype: int64

In [27]:
21184/66545

0.3183409722744008

In [84]:
pd.crosstab(df_user1.user_id,df_user1.Banana)

Banana,False,True
user_id,,
40,2,7
42,15,2
65,7,8
85,4,3
118,6,22
...,...,...
206116,4,2
206132,4,2
206146,6,4


In [85]:
ch2, p, dof, expected = chi2_contingency(pd.crosstab(df_user1.user_id,df_user1.Banana))

In [88]:
p

0.0

In [67]:
df_user1[df_user1.Banana == False].groupby("user_id").agg(withoutBanana = ("Banana","count")).reset_index()

,user_id,withoutBanana
0,40,2
1,42,15
2,65,7
3,85,4
4,118,6
...,...,...
7246,206116,4
7247,206132,4
7248,206146,6
7249,206153,9


In [68]:
df_user1[df_user1.Banana == True].groupby("user_id").agg(withBanana = ("Banana","count")).reset_index()

,user_id,withBanana
0,40,7
1,42,2
2,65,8
3,85,3
4,118,22
...,...,...
7545,206116,2
7546,206132,2
7547,206146,4
7548,206153,5


In [72]:
df_user1 = pd.merge(df_user1,df_user1[df_user1.Banana == False].groupby("user_id").agg(withoutBanana = ("Banana","count")).reset_index())

In [73]:
df_user1 = pd.merge(df_user1,df_user1[df_user1.Banana == True].groupby("user_id").agg(withBanana = ("Banana","count")).reset_index())

In [76]:
df_user1.groupby("user_id")[["withoutBanana","withBanana"]].mean().reset_index()

,user_id,withoutBanana,withBanana
0,40,2.0,7.0
1,42,15.0,2.0
2,65,7.0,8.0
3,85,4.0,3.0
4,118,6.0,22.0
...,...,...,...
7246,206116,4.0,2.0
7247,206132,4.0,2.0
7248,206146,6.0,4.0
7249,206153,9.0,5.0


In [ ]:
df_user.groupby("user_id").agg(with_banana=("Banana",""))

In [13]:
df_auf1[df_auf1.user_id == 19992].groupby("order_number").agg(coun=("order_number","count"))

,coun
order_number,
1,1
2,1
3,1


68741
270632	
606886

In [91]:
df[(df.user_id == 19992) & (df.order_number == 3 )]

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
694228,4211603,68741,24604,9,1,19992,3,0,13,8.0,1,Triscuit Roasted Garlic Baked Whole Grain Crac...,78,19,snacks,crackers,Marin,13
694229,4211607,68741,26171,1,0,19992,3,0,13,8.0,1,Lint Roller - 56 Sheets,87,17,household,more household,Marin,13
694230,4211608,68741,7353,11,0,19992,3,0,13,8.0,1,"Dill, Sea Salt & Olive Oil Crackers",78,19,snacks,crackers,Marin,13
694231,4211609,68741,42455,13,0,19992,3,0,13,8.0,1,House Blend Instant Coffee,26,7,beverages,coffee,Marin,13
694232,4211610,68741,47751,8,1,19992,3,0,13,8.0,1,Bueno,45,19,snacks,candy chocolate,Marin,13
694233,4211606,68741,33234,4,0,19992,3,0,13,8.0,1,Selects Smoked Uncured Bacon,106,12,meat seafood,hot dogs bacon sausage,Marin,13
694234,4211605,68741,36685,6,0,19992,3,0,13,8.0,1,50/50 Spring Mix & Baby Spinach,123,4,produce,packaged vegetables fruits,Marin,13
694235,4211604,68741,4367,12,0,19992,3,0,13,8.0,1,Garlic & Fine Herbs Gournay Cheese,21,16,dairy eggs,packaged cheese,Marin,13
694236,4211598,68741,13176,2,1,19992,3,0,13,8.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Marin,13
694237,4211599,68741,37766,3,1,19992,3,0,13,8.0,1,XL Emerald White Seedless Grapes,123,4,produce,packaged vegetables fruits,Marin,13


In [36]:
df[df.user_id == 19992].groupby("order_number").agg(coun=("order_number","count"))

,coun
order_number,
1,15
2,9
3,13
4,16
5,22
6,14
7,28


In [50]:
px.scatter(df_auf1.groupby("user_id").agg(coun=("tip","sum")))

In [8]:
px.scatter(df_auf1[df_auf1.reordered == 1].groupby("order_hour_of_day").agg(coun=("order_hour_of_day","count")))

In [ ]:
px.scatter(df_auf1[df_auf1.reordered == 1].groupby("order_dow").agg(coun=("order_dow","count")))

In [23]:
liste = df_auf1.groupby("user_id").agg(order_id=("order_id","min")).reset_index().order_id.to_list()
df_auf1.loc[df_auf1.order_id.isin(liste),"reordered"] = 0

In [95]:
user = df_auf1[df_auf1.reordered == 1].user_id.unique().tolist()
banana_order = df_auf1.order_id.unique().tolist()
df_user = df[df.user_id.isin(user)]

In [96]:
df_user["Banana"] = False
df_user.loc[df_user.order_id.isin(banana_order),"Banana"] = True

C:\Users\paulh\AppData\Local\Temp\ipykernel_10320\93905981.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [97]:
df_idk = df_user.groupby(["user_id","order_number"]).agg(Banana=("Banana","max"),days_since_prior_order = ("days_since_prior_order","mean")).reset_index()
df_idk["Summe"] = df_idk.days_since_prior_order

In [107]:
df_idk.head()

,user_id,order_number,Banana,days_since_prior_order,Summe,Cum
0,40,1,True,NaN,NaN,NaN
1,40,2,True,8.0,8.0,8.0
2,40,3,False,12.0,12.0,20.0
3,40,4,False,8.0,8.0,28.0
4,40,5,True,20.0,20.0,48.0


In [98]:
df_idk["Cum"] = df_idk.Summe.cumsum()

In [99]:
df_idk2 = df_idk.loc[df_idk.Banana == True]

In [100]:
df_idk2["Cumsum"] = df_idk2["Cum"].shift()

C:\Users\paulh\AppData\Local\Temp\ipykernel_10320\1643783414.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [120]:
df_idk2.loc[~(df_idk2.Cum.isna())&(df_idk2.Cumsum.isna()),"Days"] = df_idk2.Summe

In [101]:
df_idk2["Days"] = df_idk2.Cum - df_idk2.Cumsum

C:\Users\paulh\AppData\Local\Temp\ipykernel_10320\3771161101.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [102]:
df_idk2["user2"] = df_idk2.user_id.shift()

C:\Users\paulh\AppData\Local\Temp\ipykernel_10320\25003005.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [122]:
df_idk2.loc[df_idk2.user_id != df_idk2.user2,"Days"] = None

In [123]:
df_idk2.head(59)

,user_id,order_number,Banana,days_since_prior_order,Summe,Cum,Cumsum,Days,user2
0,40,1,True,NaN,NaN,NaN,NaN,NaN,NaN
1,40,2,True,8.0,8.0,8.0,NaN,8.0,40.0
4,40,5,True,20.0,20.0,48.0,8.0,40.0,40.0
5,40,6,True,15.0,15.0,63.0,48.0,15.0,40.0
6,40,7,True,21.0,21.0,84.0,63.0,21.0,40.0
7,40,8,True,7.0,7.0,91.0,84.0,7.0,40.0
8,40,9,True,14.0,14.0,105.0,91.0,14.0,40.0
15,42,7,True,9.0,9.0,155.0,105.0,NaN,40.0
16,42,8,True,11.0,11.0,166.0,155.0,11.0,42.0
26,65,1,True,NaN,NaN,NaN,166.0,NaN,42.0


In [24]:
df_idk2[df_idk2.user_id ==19992]

,user_id,order_id,Banana,days_since_prior_order,Summe,Cum,Cumsum,Days,user2
23089,19992,68741,True,8.0,8.0,213386.0,213378.0,NaN,19968.0
23093,19992,270632,True,NaN,NaN,NaN,213386.0,NaN,19992.0
23095,19992,606886,True,6.0,6.0,213512.0,NaN,NaN,19992.0


In [47]:
df_idk2[df_idk2.Days >=200]

,user_id,order_id,Banana,days_since_prior_order,Summe,Cum,Cumsum,Days,user2,Days_in_cat
360,236,592983,True,2.0,2.0,3357.0,3116.0,241.0,236.0,"(100, 500]"
2209,1739,585691,True,8.0,8.0,20062.0,19791.0,271.0,1739.0,"(100, 500]"
4336,3338,579806,True,4.0,4.0,39034.0,38825.0,209.0,3338.0,"(100, 500]"
5528,4661,592646,True,8.0,8.0,50893.0,50653.0,240.0,4661.0,"(100, 500]"
5904,4974,507075,True,4.0,4.0,54462.0,54216.0,246.0,4974.0,"(100, 500]"
...,...,...,...,...,...,...,...,...,...,...
254900,200807,355527,True,17.0,17.0,2316423.0,2316191.0,232.0,200807.0,"(100, 500]"
257902,202644,434246,True,12.0,12.0,2340463.0,2340236.0,227.0,202644.0,"(100, 500]"
259590,204172,523613,True,17.0,17.0,2355781.0,2355488.0,293.0,204172.0,"(100, 500]"
259742,204322,515845,True,4.0,4.0,2357207.0,2356951.0,256.0,204322.0,"(100, 500]"


maybe days noch in categorien

In [39]:
bins = [0,7,30,100,500]
df_idk2["Days_in_cat"] = pd.cut(df_idk2["Days"],bins)

C:\Users\paulh\AppData\Local\Temp\ipykernel_13860\3555047077.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_idk2["Days_in_cat"] = pd.cut(df_idk2["Days"],bins)


In [40]:
df_idk2

,user_id,order_id,Banana,days_since_prior_order,Summe,Cum,Cumsum,Days,user2,Days_in_cat
18,7,553865,True,2.0,2.0,199.0,NaN,NaN,NaN,NaN
21,40,74936,True,7.0,7.0,216.0,199.0,NaN,7.0,NaN
22,40,200867,True,20.0,20.0,236.0,216.0,20.0,40.0,"(7.0, 30.0]"
23,40,203217,True,14.0,14.0,250.0,236.0,14.0,40.0,"(7.0, 30.0]"
25,40,393331,True,21.0,21.0,283.0,250.0,33.0,40.0,"(30.0, 100.0]"
...,...,...,...,...,...,...,...,...,...,...
261994,206162,34456,True,12.0,12.0,2378244.0,2378175.0,NaN,206153.0,NaN
262006,206162,424565,True,27.0,27.0,2378470.0,2378244.0,226.0,206162.0,"(100.0, 500.0]"
262011,206168,109567,True,6.0,6.0,2378507.0,2378470.0,NaN,206162.0,NaN
262017,206168,465130,True,9.0,9.0,2378528.0,2378507.0,21.0,206168.0,"(7.0, 30.0]"


In [124]:
df_idk2.Days.mean()

17.81648344121374

In [125]:
px.scatter(df_idk2.groupby("Days").agg(coun=("Days","count")))

In [45]:
px.scatter(df_idk2.groupby("Days_in_cat").agg(coun=("order_id","count")))

TypeError: Object of type Interval is not JSON serializable

TypeError: Object of type Interval is not JSON serializable

TypeError: Object of type Interval is not JSON serializable

Figure({
    'data': [{'hovertemplate': 'variable=coun<br>Days_in_cat=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'coun',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'coun',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([Interval(0, 7, closed='right'), Interval(7, 30, closed='right'),
                          Interval(30, 100, closed='right'), Interval(100, 500, closed='right')],
                         dtype=object),
              'xaxis': 'x',
              'y': array([17473, 25547,  9009,  1325], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Days_in_cat'}},
               'yaxis': {'

In [11]:
df_7 = df_auf1.groupby(["user_id","reordered","county"]).agg(count=("county","count")).reset_index()

In [18]:
df_8 = df_7.groupby("user_id").agg(coun=("user_id","count")).reset_index()

In [20]:
df_8.loc[df_8.coun > 2]

,user_id,coun


In [24]:
df_1= df_auf1[df_auf1.reordered == 1]

In [44]:
df_1

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county,order_size
2792,1912138,260,13176,2,1,143768,48,3,15,4.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Los Angeles,3
6215,872041,599,13176,9,1,110568,31,0,9,7.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,San Francisco,13
12592,4563056,1252,13176,1,1,130803,7,3,9,14.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,San Francisco,8
14186,2495644,1435,13176,3,1,128799,4,5,9,15.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Orange,18
15208,4565008,1542,13176,2,1,85745,23,1,19,6.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Mariposa,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132036,3941644,606853,13176,4,1,112206,19,0,8,5.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Mariposa,18
6132178,800409,606867,13176,6,1,31632,4,0,15,7.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Alpine,24
6132363,3629290,606885,13176,1,1,32829,2,6,15,11.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Siskiyou,14
6132372,4211553,606886,13176,4,1,19992,2,6,18,6.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Marin,9


In [27]:
df_6 = df_1.groupby(["user_id","county"])[["reordered"]].sum().reset_index()

In [36]:
df_6.sort_values("reordered",ascending=False).head(30)

,user_id,county,reordered
6240,171132,San Benito,83
5517,151409,Amador,79
2128,59819,Calaveras,72
2218,62239,Kern,70
5078,139379,Modoc,70
4185,114751,Siskiyou,67
831,23170,Kern,66
3479,95471,Sutter,66
4926,134901,Kern,66
5022,137799,Santa Clara,65


In [49]:
px.scatter(df_6.groupby("county").agg(coun=("reordered","mean")))

In [47]:
df_9 = df_auf1.groupby(["user_id","reordered"])[["order_size"]].mean().reset_index()

In [48]:
df_9

,user_id,reordered,order_size
0,7,0,10.00
1,40,0,11.00
2,40,1,11.20
3,42,0,28.00
4,42,1,14.00
...,...,...,...
19430,206153,1,12.75
19431,206162,0,20.00
19432,206162,1,10.00
19433,206168,0,15.50
